In [38]:
import numpy as np
from scipy.integrate import odeint
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt
import time
from timeit import default_timer as timer
# from simCoronavirusSeattle2 import corEqs1, findBeta2
import pickle
mycolors = sns.color_palette("hls", 5)





### Loading contact matrices for Uruguay 

In [56]:
#load the contact matrices from Prem et al for the US:

urlHome = '/Users/Martin/Documents/Trabajofinal/vaccine_optimization-master/ContactMatricesUruguay/home.xlsx'
matHome = np.array(pd.read_excel(urlhome, sep=',',header=None))

urlSchool = '/Users/Martin/Documents/Trabajofinal/vaccine_optimization-master/ContactMatricesUruguay/school.xlsx'
matSchool = np.array(pd.read_excel(urlschool, sep=',',header=None))

urlWork = '/Users/Martin/Documents/Trabajofinal/vaccine_optimization-master/ContactMatricesUruguay/work.xlsx'
matWork = np.array(pd.read_excel(urlwork, sep=',',header=None))

urlOL = '/Users/Martin/Documents/Trabajofinal/vaccine_optimization-master/ContactMatricesUruguay/otherlocations.xlsx'
matOL = np.array(pd.read_excel(urlOL, sep=',',header=None))

urlAll = '/Users/Martin/Documents/Trabajofinal/vaccine_optimization-master/ContactMatricesUruguay/allLocations.xlsx'
matAll = np.array(pd.read_excel(urlAll, sep=',',header=None))


populationUY = pd.read_csv('/Users/Martin/Documents/Trabajofinal/vaccine_optimization-master/ContactMatricesUruguay/Uruguay-2019.csv')  #this was downloaded from https://www.populationpyramid.net/united-states-of-america/2020/


populationUY['total'] = populationUY['M'] + populationUY['F']
print(populationUY)



populationUYarray = np.array(populationUY['total'])
totalUYPop = np.sum(populationUYarray)

populationUYPercentages = (1/totalUYPop)*populationUYarray
print(populationUYPercentages)

      Age       M       F   total
0     0-4  121524  116289  237813
1     5-9  120179  115141  235320
2   10-14  120227  114842  235069
3   15-19  126603  121612  248215
4   20-24  130255  125951  256206
5   25-29  129993  126259  256252
6   30-34  118635  117046  235681
7   35-39  111151  111882  223033
8   40-44  116347  118466  234813
9   45-49  106646  109793  216439
10  50-54   94168  101352  195520
11  55-59   92533  102062  194595
12  60-64   81755   93781  175536
13  65-69   66567   79582  146149
14  70-74   52313   69472  121785
15  75-79   37926   57041   94967
16  80-84   24168   46000   70168
17  85-89   14052   34273   48325
18  90-94    5292   19567   24859
19  95-99    1354    7598    8952
20   100+     201    1833    2034
[0.06869771 0.06797755 0.06790505 0.07170257 0.07401095 0.07402424
 0.06808184 0.06442817 0.06783109 0.06252334 0.05648041 0.05621321
 0.05070758 0.04221847 0.03518038 0.02743339 0.02026963 0.01395978
 0.00718109 0.00258599 0.00058757]


In [60]:
def createConsistentMatrix(myMat, totalPop):
    #based on this website: https://cran.r-project.org/web/packages/socialmixr/vignettes/introduction.html
    #which assumes that the rows are participants and the columns are contacts
    [n, m] = np.shape(myMat)
    newMat = np.zeros((n, n))

    for ivals in range(n):
        for jvals in range(n):
            newMat[ivals, jvals] = (1/(2*totalPop[ivals]))*(myMat[ivals, jvals]*totalPop[ivals] + myMat[jvals, ivals]*totalPop[jvals])

    return newMat


def createAlmostSymmetricMatrix(myMat, totalPop):
    pop = np.sum(totalPop)
    [n, m] = np.shape(myMat)
    newMat = np.zeros((n,n))
    for ivals in range(n):
        for jvals in range(n):
            newMat[ivals, jvals] = myMat[ivals, jvals]*pop/totalPop[ivals]
    return newMat

def computePercentages(myMat, refMat):
    [n, m] = np.shape(myMat)
    newMat = np.zeros((n, n))
    for ivals in range(n):
        for jvals in range(n):
            newMat[ivals, jvals] = myMat[ivals, jvals]/refMat[ivals, jvals]
    return newMat


populationUY16groups = np.concatenate([populationUYarray[0:15], [np.sum(populationUYarray[15:])]])
print((populationUY16groups))
pop75_80 = populationUYarray[15]
print(pop75_80)
pop80plus = np.sum(populationUYarray[16:])
print(pop80plus)



populationUY16groupsPercentages = (1/totalUYPop)*populationUY16groups
print(populationUY16groupsPercentages)


myNewMatAll2 = createConsistentMatrix(matAll.transpose(), populationUY16groups)

myNewMatHome2 = createConsistentMatrix(matHome.transpose(), populationUY16groups)

myNewMatSchool2 = createConsistentMatrix(matSchool.transpose(), populationUY16groups)

myNewMatWork2 = createConsistentMatrix(matWork.transpose(), populationUY16groups)

myNewMatOL2 = createConsistentMatrix(matOL.transpose(), populationUY16groups)




mymats2 = {"all": myNewMatAll2, 'home': myNewMatHome2, 'school': myNewMatSchool2, 'work': myNewMatWork2, 'otherloc': myNewMatOL2}

#####save the results in a pickle file
today = time.strftime("%d%b%Y", time.localtime())
myfilename = 'consistentMatricesUY_polymodMethod' + today + '.pickle'
myfile = open(myfilename, 'wb')
pickle.dump(mymats2, myfile)
myfile.close()





[237813 235320 235069 248215 256206 256252 235681 223033 234813 216439
 195520 194595 175536 146149 121785 249305]
94967
154338
[0.06869771 0.06797755 0.06790505 0.07170257 0.07401095 0.07402424
 0.06808184 0.06442817 0.06783109 0.06252334 0.05648041 0.05621321
 0.05070758 0.04221847 0.03518038 0.07201744]
